# **Notebook de transfert des donnees du site nba.com depuis le site ou un fichier intermediaire**

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys, os, re, time #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct

from datetime import datetime
from TelechargementDonnees import JoueursSiteNba, DriverFirefox, gererCookieNba, gererCookieTtfl,simplifierNomJoueur, JourneeSiteNba, Blessures, JoueursChoisisTtfl
from TeleversementJourneeBdd import JourneeBdd, insererBlessesInconnusMatchBefore, miseAJourBlessesBdd, insererBlessesInconnusPasMatchBefore, transfertCalendrier, televerserJourneeSiteNba
from Ihm.Initialiser_donnees_IHM import lastDates
from sqlalchemy import MetaData
from datetime import timedelta, date

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
from selenium.webdriver.support.ui import Select
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

In [3]:
import Ihm.MainWindow as mw

In [4]:
pd.set_option('display.max_rows', 500)

# 1. Données de joueurs 
Il s'agit de transfere dans le cas des donnees de joueuer telechargees en amont, ou d'ajouter des joueurs non presents au moment d'insertion de match

>## 1.1 Enumerations des types de position
A creer a la main

In [ ]:
with ct.ConnexionBdd('basket','maison') as c : 
    pd.DataFrame({'id_position_terrain':['F', 'G', 'C', 'C-F', 'F-G', 'G-F', 'F-C','NC'], 
              'nom_position_terrain':['Forward ; 3 ; ailier', 'Guard ; 1 ou 2  ; meneur ou arriere', 
                                      'Center ; 5 ; Pivot' ,'Power Forward ; 4 ; ailier Fort', 
                                      'Shooting Guard ; 2 ; arriere', 'Shooting Guard ; 2 ; arriere',
                                      'Power Forward ; 4 ; ailier Fort', 'Non connue']}).to_sql(
        'enum_position_terrain', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

> ## 1.2 Donnees chargees depuis la page internet
Là ce n'est que le premier import, on ne gère pas la mise à jour

In [11]:
#creation de la classe si import depuis le net
#pagejoueuer=JoueursSiteNba(urlPageJoueurs='https://www.nba.com/player/1630173/precious-achiuwa',typeExport='One' ) #joueur unique
pagejoueuer=JoueursSiteNba() #joueurs multiples

NoSuchElementException: Message: Unable to locate element: //td[@class='primary text PlayerList_primaryCol__1tWZA']


In [10]:
pagejoueuer.dfJoueurs

AttributeError: type object 'JoueursSiteNba' has no attribute 'dfJoueurs'

In [39]:
#si import depuis json
Joueur=pd.read_json(r'C:\Users\martin.schoreisz\git\Basket\Basket\data\Joueurs_saison_2020-2021\joueurs.json')
#correction format date
Joueur.date_entree_nba=Joueur.date_entree_nba.apply(lambda x : pd.to_datetime(x,unit='ms'))
Joueur.date_naissance=Joueur.date_naissance.apply(lambda x : pd.to_datetime(x,unit='ms'))
#si besoin correction position bizrre car inconnues sur le net (normalenet pris en cmpte en amont dans nouvelle versions)
Joueur.loc[~Joueur.id_position_terrain.isin(['F', 'G', 'C', 'C-F', 'F-G', 'G-F', 'F-C','NC']),'id_position_terrain']='NC'

In [40]:
with ct.ConnexionBdd('basket','maison') as c : 
    #remmplacer pagejoueuer.dfJoueurs par Joueur si import Json
    Joueur[['nom','id_position_terrain','taille', 'poids','date_entree_nba','nom_simple']].to_sql(
        'joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 2. Données d'équipe
On utilise les données de joueurs pour faire la liste des equipes, et on complete avec les divisions et conférence, puis on transfere

In [41]:
dicoNom={'MIA':'Miami Heat','MIL':'Milwaukee Bucks','NOP':'New-Orleans Pelicans','SAS':'San-Antonio Spurs',
'PHX':'Phoenix','MEM':'Memphis Grizzlies',    'BKN':'Brooklyn Nets','ORL':'Orlando Magic','LAL':'LosAngeles Lakers','POR':'Portland TrailBlazzers','TOR':'Toronto Raptors','CHI':'Chicago Bulls',
'OKC':'Oklahoma City Thunder','WAS':'Washington Wizards','UTA':'Utah Jazz','SAC':'Sacramento Kings','CHA':'Charlotte Hornets','NYK':'New-York Knicks','DEN':'Denver Nuggets',
'LAC':'Los Angeles Clippers','GSW':'Golden State Warriors','MIN':'Minessota TimberWolves','DET':'Detroit Pistons','DAL':'Dallas Mavericks',
'IND':'Indiana Pacers','ATL':'Atlanta Hawks','CLE':'Cleveland Cavaliers','PHI':'Philadelphia Sixers','BOS':'Boston Celtics','HOU':'Houston Rockets'}
list_equipe=dicoNom.keys()

In [42]:
dicoDivision={'Central Division': ['MIL', 'CHI', 'IND', 'CLE', 'DET'],'Atlantic Division': ['BOS', 'BKN', 'NYK', 'TOR', 
'PHI'],'Southeast Division' : ['MIA', 'ORL', 'WAS', 'CHA', 'ATL'],'Southwest Division': ['SAS', 'HOU', 'DAL', 'NOP', 
'MEM'],'Northwest Division' : ['DEN', 'MIN', 'OKC', 'POR', 'UTA'],'Pacific Division' : ['LAC', 'LAL', 'PHX', 'GSW', 'SAC']}
listDivision=[k for e in list_equipe for k,v in dicoDivision.items()  if e in v] 

In [43]:
dicoConference={'Ouest' : ['SAS', 'HOU', 'DAL', 'NOP', 'MEM','DEN', 'MIN', 'OKC', 'POR', 'UTA','LAC', 'LAL', 'PHX', 
    'GSW', 'SAC'],'Est':['MIL', 'CHI', 'IND', 'CLE', 'DET','BOS', 'BKN', 'NYK', 'TOR', 'PHI','MIA', 
                         'ORL', 'WAS', 'CHA', 'ATL']}

listConference=[k for e in list_equipe for k,v in dicoConference.items()  if e in v] 
listNom=[v for k,v in dicoNom.items() for e in list_equipe if k==e] 

In [44]:
with ct.ConnexionBdd('basket','boulot') as c : 
    pd.DataFrame({'id_equipe':list_equipe,'nom_equipe': listNom,
              'conference':listConference, 'division':listDivision}).to_sql(
        'equipe', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 3. Données de match
Lecture et transfert des données du site vers la bdd

In [3]:
dateMatchRecentBdd, dateCalendrierRecent=lastDates()[0:2]

In [11]:
dateMatchRecentBdd, dateCalendrierRecent

(datetime.date(2021, 3, 28), datetime.date(2021, 4, 12))

In [4]:
listeDate=[d.strftime('%Y-%m-%d') for d in pd.date_range(start='2021-03-31',
                                                                    periods=1)]
for e,j in  enumerate(listeDate): 
    print(j)
    journee=JourneeBdd(j)
    journee.creerAttributsGlobaux()
    journee.exporterVersBdd()

2021-03-31
0 https://www.nba.com/game/por-vs-det-0022000721/box-score#box-score
1 https://www.nba.com/game/mia-vs-ind-0022000722/box-score#box-score
2 https://www.nba.com/game/dal-vs-bos-0022000723/box-score#box-score
3 https://www.nba.com/game/hou-vs-bkn-0022000724/box-score#box-score
4 https://www.nba.com/game/uta-vs-mem-0022000725/box-score#box-score
5 https://www.nba.com/game/nyk-vs-min-0022000726/box-score#box-score
6 https://www.nba.com/game/tor-vs-okc-0022000727/box-score#box-score
7 https://www.nba.com/game/sac-vs-sas-0022000728/box-score#box-score
8 https://www.nba.com/game/mil-vs-lal-0022000729/box-score#box-score
9 https://www.nba.com/game/chi-vs-phx-0022000730/box-score#box-score


# 4. exemple de blessure
le joueur Markelle Fultz s'est fait une rupture des ligaments croisé, exemple d'insertion dans la table blessure

In [ ]:
nomJoueurBlesse='Markelle Fultz'
idTypeBlessure=1
dateBlessure='2021-06-01'
requeteBlessure = f"""insert into donnees_source.blessure(id_joueur,id_type_blessure,date_blessure) 
             select id_joueur,{idTypeBlessure},{dateBlessure} from donnees_source.joueur where lower(nom)=lower{nomJoueurBlesse}"""
with ct.ConnexionBdd('basket','maison') as c : 
    c.execute(c.sqlAlchemyConn,requeteBlessure)

In [46]:
#dfsInjuries=Blessures().dfInjuries
with ct.ConnexionBdd('basket','maison') as c : 
    #miseAJourBlessesBdd(dfsInjuries,c.sqlAlchemyConn)
    #insererBlessesInconnusMatchBefore(dfsInjuries,c.sqlAlchemyConn)
    insererBlessesInconnusPasMatchBefore(dfsInjuries,c.sqlAlchemyConn)

# 5. TTFL : inserer les joueurs choisis
Inserer dans la base les joueurs deja choisis en ttfl ;  
Pour le moment je télécharge manuellement les donnees depuis trashtalk
à terme on pourra telecaharger auto (attention, gestion mot de passe) ou remplir directement depuis une liste exterieure

In [5]:
JoueurChoisis=JoueursChoisisTtfl(2)

connexion a etablir


In [6]:
JoueurChoisis.dfJoueurChoisiId.sort_values('Date')

,Date,Joueur,Pts,Reb,Ast,Stl,Blk,Ftm,Fgm,Fg3m,Malus,Score,nom_simple,id_joueur
8,2021-10-19,Stephen Curry,21,10,10,3,0,9,5,2,26,34,stephencurry,189
7,2021-10-20,Joel Embiid,22,6,5,1,1,4,8,2,13,36,joelembiid,216
6,2021-10-21,Trae Young,19,2,14,1,0,5,6,2,18,31,traeyoung,577
5,2021-10-22,Julius Randle,21,10,7,2,0,1,9,2,16,36,juliusrandle,471
4,2021-10-23,Karl Anthony Towns,25,4,2,0,3,2,10,3,18,31,karlanthonytowns,533
3,2021-10-24,Jayson Tatum,31,9,2,1,1,3,12,4,21,42,jaysontatum,518
2,2021-10-25,LaMelo Ball,25,5,9,0,0,4,7,7,25,32,lameloball,105
1,2021-10-26,Luka Doncic,26,14,7,2,1,5,10,1,24,42,lukadoncic,201
0,2021-10-27,Anthony Davis,30,8,2,1,0,6,12,0,14,45,anthonydavis,191


In [7]:
JoueurChoisis.dfJoueurAInserer

,Date,Joueur,Pts,Reb,Ast,Stl,Blk,Ftm,Fgm,Fg3m,Malus,Score,nom_simple,id_joueur
0,2021-10-27,Anthony Davis,30,8,2,1,0,6,12,0,14,45,anthonydavis,191
1,2021-10-26,Luka Doncic,26,14,7,2,1,5,10,1,24,42,lukadoncic,201
2,2021-10-25,LaMelo Ball,25,5,9,0,0,4,7,7,25,32,lameloball,105
3,2021-10-24,Jayson Tatum,31,9,2,1,1,3,12,4,21,42,jaysontatum,518
4,2021-10-23,Karl Anthony Towns,25,4,2,0,3,2,10,3,18,31,karlanthonytowns,533
5,2021-10-22,Julius Randle,21,10,7,2,0,1,9,2,16,36,juliusrandle,471
6,2021-10-21,Trae Young,19,2,14,1,0,5,6,2,18,31,traeyoung,577
7,2021-10-20,Joel Embiid,22,6,5,1,1,4,8,2,13,36,joelembiid,216
8,2021-10-19,Stephen Curry,21,10,10,3,0,9,5,2,26,34,stephencurry,189


In [8]:
JoueurChoisis.transfertBdd()

# 6. tranferer le calendrier des matchs a venir
en fonction d'une date et duree.  
il reste a mettre en place un systeme qui checke en amont les donnees deja presente et delete celle qui ne sont plus dans la journee (ou reportee, a verifier), et qui insere de nouvelle

In [9]:
transfertCalendrier(2,'20/10/2021',15)

debut
avant driver
apres driver


# 7.Test

In [10]:
test=mw.WorkerCalendrier(1,'2021-04-13',2)

In [11]:
test.run()

debut run
avant tranbsmission
avant calendrier
